In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam, SGD
# sys.version

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=sys.maxsize)

### Loading the data

In [41]:
input_texts = []
target_texts = []

In [4]:
for line in open("../poetry_generator/data/robert_frost.txt"):
    line = line.rstrip()
    if not line:
        continue
        
    input_line = '<sos>' + line
    target_line = line + '<eos>'
    
    input_texts.append(input_line)
    target_texts.append(target_line)

In [5]:
all_lines = input_texts + target_texts
all_lines[:15]

['<sos>Two roads diverged in a yellow wood,',
 '<sos>And sorry I could not travel both',
 '<sos>And be one traveler, long I stood',
 '<sos>And looked down one as far as I could',
 '<sos>To where it bent in the undergrowth;',
 '<sos>Then took the other, as just as fair,',
 '<sos>And having perhaps the better claim',
 '<sos>Because it was grassy and wanted wear,',
 '<sos>Though as for that the passing there',
 '<sos>Had worn them really about the same,',
 '<sos>And both that morning equally lay',
 '<sos>In leaves no step had trodden black.',
 '<sos>Oh, I kept the first for another day!',
 '<sos>Yet knowing how way leads on to way',
 '<sos>I doubted if I should ever come back.']

In [6]:
len(all_lines)

2872

### Converting sentences into integers and word to integer mapping
#### Tokenizer 1) Splits into individual tokens/words 2) each word is converted into an integer index for mapping 

In [7]:
from keras.preprocessing.text import Tokenizer

MAX_VOCAB_SIZE = 3000
MAX_SEQUENCE_LENGTH = 100

In [8]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='')
tokenizer.fit_on_texts(all_lines)
input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)

target_sequences[:15]

[[133, 571, 572, 7, 3, 573],
 [4, 574, 5, 66, 28, 984],
 [4, 24, 25, 985, 163, 5],
 [4, 181, 69, 25, 14, 137, 14, 5],
 [2, 39, 8, 986, 7, 1],
 [130, 197, 1, 575, 14, 73, 14],
 [4, 145, 305, 1, 260],
 [440, 8, 12, 576, 4, 244],
 [155, 14, 13, 11, 1, 577, 2280],
 [21, 579, 50, 415, 122, 1],
 [4, 153, 11, 987, 988, 2281],
 [7, 989, 44, 581, 21, 990, 2282],
 [941, 5, 261, 1, 262, 13, 323],
 [191, 992, 106, 80, 993, 15, 2],
 [5, 994, 29, 5, 126, 154, 86, 2283]]

In [9]:
print(input_sequences[:15])

max_seq_len_from_data = max(len(s) for s in input_sequences)
print('Maximum sequence length', max_seq_len_from_data)


[[413, 571, 572, 7, 3, 573, 808], [16, 574, 5, 66, 28, 984, 153], [16, 24, 25, 985, 163, 5, 258], [16, 181, 69, 25, 14, 137, 14, 5, 66], [59, 39, 8, 986, 7, 1, 2408], [259, 197, 1, 575, 14, 73, 14, 2409], [16, 145, 305, 1, 260, 2410], [2411, 8, 12, 576, 4, 244, 2412], [414, 14, 13, 11, 1, 577, 58], [578, 579, 50, 415, 122, 1, 809], [16, 153, 11, 987, 988, 580], [102, 989, 44, 581, 21, 990, 991], [2413, 5, 261, 1, 262, 13, 323, 2414], [810, 992, 106, 80, 993, 15, 2, 80], [30, 994, 29, 5, 126, 154, 86, 324]]
Maximum sequence length 11


#### word to integer mapping

In [10]:
word_to_index = tokenizer.word_index
print('unique tokens =',len(word_to_index))
# assert('<sos>' in word_to_index)
# assert('<eos>' in word_to_index)

unique tokens = 4614


### Padding sequences --> by adding padding tokens, all sequences of varying lengths have same length so that they can be effectively processed by the model

In [11]:
max_seq_len = min(max_seq_len_from_data, MAX_SEQUENCE_LENGTH)
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='post')
target_sequences = pad_sequences(target_sequences, maxlen=max_seq_len, padding='post')

In [12]:
print('Input sequence is {} with the size of {}' .format(input_sequences[0], input_sequences.shape[1]))
print('Target sequence is {} with the size of {}' .format(target_sequences[0], target_sequences.shape[1]))

Input sequence is [413 571 572   7   3 573 808   0   0   0   0] with the size of 11
Target sequence is [133 571 572   7   3 573   0   0   0   0   0] with the size of 11


### Load pre-trained word vectors with GloVe and word --> vec mapping

In [13]:
path2glove = os.getcwd() + '/glove/glove.6B.50d.txt'

In [14]:
word2vec = {}
with open(path2glove) as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
        
    print('Found {} word vectors.' .format(len(word2vec)))

Found 400000 word vectors.


In [15]:
from itertools import islice

word2vec_first_10 = dict(islice(word2vec.items(), 3))

print(word2vec_first_10)

{'the': array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32), ',': array([ 0.013441,  0.23682 , -0.16899 ,  0.40951 ,  0.63812 ,  0.47709 ,
       -0.42852 , -0.55641 , -0.364   , -0.23938 ,  0.13001 , -0.063734,
       -0.39575 , -0.48162 ,  0.23291 ,  0.090201, -0.13324 ,  0.078639,
       -0.41634 , -0.1542

### Embedding matrix

In [16]:
print('Filling pretrained embeddings...')

Filling pretrained embeddings...


In [17]:
EMBEDDING_DIM = 50

num_words = min(MAX_VOCAB_SIZE, len(word_to_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_to_index.items():
    if i < MAX_VOCAB_SIZE:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [20]:
print(embedding_matrix.shape)
embedding_matrix

(3000, 50)


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.41800001,  0.24968   , -0.41242   , ..., -0.18411   ,
        -0.11514   , -0.78580999],
       [ 0.68046999, -0.039263  ,  0.30186   , ..., -0.073297  ,
        -0.064699  , -0.26043999],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

### One-hot encoded target 

#### The target is a sequence, so can not pass a sparse categorical cross entropy loss function to keras as I have multiple targets per input

In [21]:
print(input_sequences.shape)
print(max_seq_len)
print(num_words)

(1436, 11)
11
3000


In [29]:
one_hot_targets = np.zeros((len(input_sequences), max_seq_len, num_words))
for i, target_sequence in enumerate(target_sequences):
    for t, word in enumerate(target_sequence):
        if word > 0:
            one_hot_targets[i, t, word] = 1

In [43]:
one_hot_targets[3]

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

### Load pre-trained word embeddings into an Embedding layer

In [46]:
embedding_layer = Embedding(num_words, 
                            EMBEDDING_DIM, 
                            weights=[embedding_matrix])

### Create an LSTM network with a single LSTM

#### Building a model...

In [59]:
from keras.layers import Dense, Embedding, Input, LSTM

LATENT_DIM = 25

In [60]:
input_ = Input(shape = (max_seq_len,))
input_h = Input(shape = (LATENT_DIM,))
input_c = Input(shape = (LATENT_DIM,))

In [63]:
x = embedding_layer(input_)

# LSTM has to predict rest of the sequence and the output must be a sequence, hence return_sequences=True
lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
x, _, _ = lstm(x, initial_state=[input_h, input_c])

dense = Dense(num_words, activation='softmax')
output = dense(x)

In [67]:
model = Model([input_, input_h, input_c], output)
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(learning_rate=0.01),
              metrics = ['accuracy'])